In [1]:
import requests
from config import req_headers, base_url, focus_league, local_host, local_name, local_port, local_user
from creds import local_db_password
import psycopg2
import time
import json

In [2]:
conn = psycopg2.connect(dbname=local_name, user =local_user, host=local_host, password=local_db_password, port =local_port)
cur = conn.cursor()

In [3]:
endpoint = 'fixtures/players'
url = base_url + endpoint
season = 2024

In [4]:
fixture_list = []
fetch_teams_sql = f"""select id from ffl.fixtures where tracking_status = 'posted' and season = '{season}' and date < now()"""
cur.execute(fetch_teams_sql)
for i in cur.fetchall():
    fixture_list.append(i[0])

In [5]:
fixture_insert_sql = """
insert into ffl_staging.query_data (
    job_id,
    query_scope,
    response_data,
    status
)
values
(%s,%s,%s, %s)
on conflict do nothing
"""

In [6]:
def parser(data=None, id_counter = None):
    job_id = id_counter
    query_scope = endpoint
    response_data = data


    return job_id, query_scope, json.dumps(response_data), 'loaded'

In [7]:
def fetch_response(params=None, page = None):
    if page == None:
        req = requests.get(url, params=params, headers=req_headers)

        return req.json()
    else:
        params['page'] = page
        req = requests.get(url, params=params, headers=req_headers)

        return req.json()

def load_responses(response_data=None, count_pass=None):
    cur.execute(fixture_insert_sql, (parser(response_data, count_pass)))

In [8]:
update_sql = """
update ffl.fixtures
set tracking_status = 'player_queried'
where id = %s
"""

In [9]:
for fixture in fixture_list:
    # time.sleep(30)
    print(f'starting new fixture id {fixture}')
    call_params = {
        'fixture':fixture
    }
    counter = 1
    response = fetch_response(call_params)
    load_responses(response, counter)
    conn.commit()
    paging_needed = True
    
    while paging_needed == True:
        # time.sleep(6.777)
        paging_current = response['paging']['current']
        paging_total = response['paging']['total']
        if paging_current == paging_total:
            paging_needed = False
            # print('paging not needed')
        elif paging_current != paging_total:
            next_page = paging_current + 1
            response = fetch_response(call_params, next_page)
            load_responses(response, counter)
            conn.commit()
        counter += 1
            # print(f'completed paging. page #{next_page}')
    cur.execute(update_sql, (fixture,))
    conn.commit()
    print(f'completed fixture with {counter} pages')

starting new fixture id 1208021
completed fixture with 2 pages
starting new fixture id 1208022
completed fixture with 2 pages
starting new fixture id 1208023
completed fixture with 2 pages
starting new fixture id 1208024
completed fixture with 2 pages
starting new fixture id 1208025
completed fixture with 2 pages
starting new fixture id 1208026
completed fixture with 2 pages
starting new fixture id 1208027
completed fixture with 2 pages
starting new fixture id 1208028
completed fixture with 2 pages
starting new fixture id 1208029
completed fixture with 2 pages
starting new fixture id 1208030
completed fixture with 2 pages
starting new fixture id 1208031
completed fixture with 2 pages
starting new fixture id 1208032
completed fixture with 2 pages
starting new fixture id 1208033
completed fixture with 2 pages
starting new fixture id 1208034
completed fixture with 2 pages
starting new fixture id 1208035
completed fixture with 2 pages
starting new fixture id 1208036
completed fixture with 